In [2]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import json
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.5.2515.0


## 1) Create your Instrument Universe

Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here LUSID Knowledge Base: What is an Instrument?.

As part of this definition you will also attach two alias identifiers to your instruments. Read more about alias identifiers here LUSID Knowledge Base: What is an Alias Identifier?.

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here LUSID Knowledge Base: Upsert.

For further usage of the upsert instruments API call refer to the LUSID API Docs: Upserting Instruments.

Run the cell below to upsert your instruments into LUSID

In [4]:
def upsert_instruments(instrument_universe=None, identifier_columns=None):
    """
    This function upserts instruments from a dataframe 
    
    Keyword arguments:
    instrument_universe (Pandas DataFrame) -- The imported instrument universe
    identifer_columns (Tuple (string, string) -- The identifier name in CSV file 
    and identifier name in LUSID
    
    Returns:
    N/A
    """
    
    # Initialise your batch upsert request
    batch_upsert_request = {}

    # Using your instrument universe create your batch request
    for index, instrument in instrument_universe.iterrows():
    
        # Create your identifiers
        identifiers = {}
        for identifier in identifier_columns:
            identifiers[identifier[1]] = models.InstrumentIdValue(
                value=instrument[identifier[0]])

        # Build your request and add it to the dictionary
        batch_upsert_request[instrument['InstrumentName']] = models.InstrumentDefinition(
            name=instrument['InstrumentName'],
            identifiers=identifiers
        )
        
    # Call LUSID to upsert your instrument defintions
    instrument_response = client.instruments.upsert_instruments(
        request_body=batch_upsert_request)
    
    # Pretty print the response
    prettyprint.instrument_response(instrument_response)

In [5]:
equity_source = pd.read_csv('data/multiplesystems-instruments-equities.csv')
equity_source.head()

,InstrumentName,ClientInternal,Currency,Isin,Figi,ExchangeCode,CountryIssue,Ticker,MarketSector,SecurityType,Coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN


In [7]:
upsert_instruments(
    instrument_universe=equity_source, 
    identifier_columns=[
        ('ClientInternal', 'ClientInternal'),
        ('Isin', 'Isin'),
        ('Figi', 'Figi'),
        ('Ticker', 'Ticker')
    ])

Instrument Successfully Upserted: WPP_LondonStockEx_WPP
ClientInternal ID: imd_34536734
LUSID Instrument ID: LUID_CQ1YT89E


Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
ClientInternal ID: imd_34634534
LUSID Instrument ID: LUID_MV2JFLHO


Instrument Successfully Upserted: Sage_LondonStockEx_SGE
ClientInternal ID: imd_23536673
LUSID Instrument ID: LUID_QR9GOEKU


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
ClientInternal ID: imd_35345345
LUSID Instrument ID: LUID_L0HPF4AZ


Instrument Successfully Upserted: Glencore_LondonStockEx_GLEN
ClientInternal ID: imd_34534555
LUSID Instrument ID: LUID_8UPBY0FY


Instrument Successfully Upserted: MicroFocus_LondonStockEx_MCRO
ClientInternal ID: imd_34567338
LUSID Instrument ID: LUID_8AQU4CVN


Instrument Successfully Upserted: BP_LondonStockEx_BP
ClientInternal ID: imd_43535553
LUSID Instrument ID: LUID_W5IV22G2


Instrument Successfully Upserted: JustEat_LondonStockEx_JE
ClientInternal ID: imd_35436366
LUSID Instrument ID: LUID_6H

In [8]:
bond_source = pd.read_csv('data/multiplesystems-instruments-bonds.csv')
bond_source.head()

,InstrumentName,ClientInternal,Currency,Isin,Figi,ExchangeCode,CountryIssue,Ticker,MarketSector,SecurityType,Coupon
0,UKGiltTreasury_2.0_2025,imd_34534536,GBP,GB00BTHH2R79,NaN,LN,united_kingdom,UKT 2 09/07/25,govt,uk_gilt_stock,2.00
1,UKGiltTreasury_3.5_2045,imd_54234532,GBP,GB00BN65R313,NaN,LN,united_kingdom,UKT 3.5 01/22/45,govt,uk_gilt_stock,3.50
2,UKGiltTreasury_3.75_2021,imd_34643653,GBP,GB00B4RMG977,NaN,LN,united_kingdom,UKT 3.75 09/07/21,govt,uk_gilt_stock,3.75
3,UKGiltTreasury_4.5_2034,imd_34534534,GBP,GB00B52WS153,NaN,LN,united_kingdom,UKT 4.5 09/07/34,govt,uk_gilt_stock,4.50
4,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,NaN,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.00


In [9]:
upsert_instruments(
    instrument_universe=bond_source, 
    identifier_columns=[
        ('ClientInternal', 'ClientInternal'),
        ('Isin', 'Isin'),
        ('Ticker', 'Ticker')
    ])

Instrument Successfully Upserted: UKGiltTreasury_2.0_2025
ClientInternal ID: imd_34534536
LUSID Instrument ID: LUID_U9KBNZ7I


Instrument Successfully Upserted: UKGiltTreasury_4.5_2034
ClientInternal ID: imd_34534534
LUSID Instrument ID: LUID_LVDEEG2Z


Instrument Successfully Upserted: USTreasury_6.875_2025
ClientInternal ID: imd_34534539
LUSID Instrument ID: LUID_FFGAS6UW


Instrument Successfully Upserted: USTreasury_2.00_2021
ClientInternal ID: imd_34535347
LUSID Instrument ID: LUID_5060M7RV


Instrument Successfully Upserted: UKGiltTreasury_3.5_2045
ClientInternal ID: imd_54234532
LUSID Instrument ID: LUID_Y3OW5T1Z


Instrument Successfully Upserted: UKGiltTreasury_3.75_2021
ClientInternal ID: imd_34643653
LUSID Instrument ID: LUID_DAQZIXFK


6  instruments upserted successfully
0  instrument upsert failures


In [11]:
option_source = pd.read_csv('data/multiplesystems-instruments-options.csv')
option_source.head()

,InstrumentName,ClientInternal,Currency,Isin,Figi,FigiComposite,Sedol,ExchangeCode,CountryIssue,Ticker,MarketSector,SecurityType,Coupon
0,October 19 Calls on AMZN US,imd_84634539,USD,NaN,BBG00NFXK409,BBG00NFXK409,NaN,UN,united_states_america,AMZN 10/18/19 C1365,equity,equity_option,NaN
1,October 19 Puts on AAPL US,imd_85345347,USD,NaN,BBG00NBRV912,BBG00NBRV912,NaN,UN,united_states_america,AAPL 10/18/19 P140,equity,equity_option,NaN
2,September 19 Calls on BP/ LN,imd_83535553,GBP,NaN,BBG00M2Z8958,NaN,NaN,LN,united_kingdom,BPA 09/20/19 C570,equity,equity_option,NaN


In [27]:
upsert_instruments(
    instrument_universe=option_source, 
    identifier_columns=[
        ('ClientInternal', 'ClientInternal'),
        ('Figi', 'Figi'),
        ('Ticker', 'Ticker')
    ])

Instrument Successfully Upserted: September 19 Calls on BP/ LN
ClientInternal ID: imd_83535553
LUSID Instrument ID: LUID_RJTRD1LM


Instrument Successfully Upserted: October 19 Puts on AAPL US
ClientInternal ID: imd_85345347
LUSID Instrument ID: LUID_4MZTEXSJ


Instrument Successfully Upserted: October 19 Calls on AMZN US
ClientInternal ID: imd_84634539
LUSID Instrument ID: LUID_0F2W8COI


3  instruments upserted successfully
0  instrument upsert failures


## 2) Set up a Scope for each Source System

In [13]:
bonds_scope = 'bonds_system_{}'.format(str(uuid.uuid4())[:4])
equities_scope = 'equities_system_{}'.format(str(uuid.uuid4())[:4])
options_scope = 'options_system_{}'.format(str(uuid.uuid4())[:4])

scopes = [bonds_scope, equities_scope, options_scope]

prettyprint.heading('Bonds Scope', bonds_scope)
prettyprint.heading('Equities Scope', bonds_scope)
prettyprint.heading('Options Scope', bonds_scope)

Bonds Scope: bonds_system_764d
Equities Scope: bonds_system_764d
Options Scope: bonds_system_764d


## 3) Create your Portfolio in Each Scope

In [14]:
# Set the code of your portfolio
portfolio_code = 'Global-Strategies'

# Set the creation date of your portfolio 
portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name='Global Strategies Fund',
    code=portfolio_code,
    base_currency='GBP',
    description=None,
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=None,
    properties=None)


for scope in scopes:
    
    # Call LUSID to create your portfolio
    response = client.transaction_portfolios.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=request)

    # Pretty print the response
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: bonds_system_764d
Code: Global-Strategies
Portfolio Effective From: 2016-05-29 15:17:07.039155+00:00
Portfolio Created On: 2019-04-16 15:17:05.985026+00:00

Portfolio Created
Scope: equities_system_eaca
Code: Global-Strategies
Portfolio Effective From: 2016-05-29 15:17:07.039155+00:00
Portfolio Created On: 2019-04-16 15:17:06.953297+00:00

Portfolio Created
Scope: options_system_2e06
Code: Global-Strategies
Portfolio Effective From: 2016-05-29 15:17:07.039155+00:00
Portfolio Created On: 2019-04-16 15:17:07.311082+00:00



## 3) Set your Initial Holdings

In [15]:
def load_holdings(holdings, scope, code, holdings_effective_date, instrument_identifier):
    """
    This function creates an identifier set of unique and alias identifiers
    
    Keyword arguments:
    holdings (Pandas Dataframe) -- Details of holdings
    scope (string) -- The scope of the portfolio to add holdings too
    code (string) -- The code of the portfolio to add holdings too 
    holdings_effective_date (datetime.datetime) -- The date from which holdings are effective
    instrument_identifier (string) -- The unique instrument identifier to use for adding the holdings
    
    Returns:
    """
    
    # Iterate the portfolios in the holdings CSV, note in this case you only have one
    for portfolio in holdings['PortfolioCode'].unique():
        # Initialise a list to hold your adjustments
        holding_adjustments = []

        # Iterate over the holdings in each portfolio
        for index, holding in holdings.loc[holdings['PortfolioCode'] == portfolio].iterrows():

            # Set your instrument identifiers based on whether or not instrument is cash
            if 'Cash' in holding['InstrumentName']:
                identifier_key = 'Instrument/default/Currency'
                identifer = holding['InstrumentName'].split('_')[0]
            else:
                identifier_key = 'Instrument/default/{}'.format(instrument_identifier)
                identifer = holding[instrument_identifier]

            # Create your holding adjustment and append it to your list
            holding_adjustments.append(
                models.AdjustHoldingRequest(
                    instrument_identifiers={
                        identifier_key: identifer},
                    tax_lots=[
                        models.TargetTaxLotRequest(
                            units=holding['Quantity'],
                            cost=models.CurrencyAndAmount(
                                amount=holding['Quantity'] * holding['Price'],
                                currency=holding['Currency']),
                            portfolio_cost=holding['Quantity'] * holding['Price'],
                            price=holding['Price'])
                    ]
                )
            )

        # Call LUSID to set your initial holdings
        response = client.transaction_portfolios.set_holdings(
            scope=scope,
            code=code,
            effective_at=holdings_effective_date,
            adjust_holding_request=holding_adjustments)

        # Pretty print our response from LUSID
        prettyprint.set_holdings_response(response, scope, portfolio)

In [16]:
holdings = pd.read_csv('data/multiplesystems-holdings-bonds.csv')
holdings.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi,ClientInternal
0,Global-Strategies,UKGiltTreasury_2.0_2025,405589,106.637,GBP,NaN,imd_34534536
1,Global-Strategies,UKGiltTreasury_3.5_2045,266169,134.433,GBP,NaN,imd_54234532
2,Global-Strategies,UKGiltTreasury_3.75_2021,661713,108.126,GBP,NaN,imd_34643653
3,Global-Strategies,UKGiltTreasury_4.5_2034,77481,140.572,GBP,NaN,imd_34534534
4,Global-Strategies,USTreasury_2.00_2021,1440244,97.900,USD,NaN,imd_34535347


In [17]:
# Make the holdings effective from two days ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=2)

load_holdings(holdings=holdings, 
              scope=bonds_scope, 
              code=portfolio_code, 
              holdings_effective_date=holdings_effective_date, 
              instrument_identifier='ClientInternal')

Holdings Successfully Set for Portfolio
Scope: bonds_system_764d
Code: Global-Strategies
Holdings Effective From: 2019-04-14 15:17:38.549387+00:00
Holdings Created On: 2019-04-16 15:17:38.931141+00:00



In [18]:
holdings = pd.read_csv('data/multiplesystems-holdings-equities.csv')
holdings.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi
0,Global-Strategies,GBP_Cash,5557333,1.000,GBP,NaN
1,Global-Strategies,Glencore_LondonStockEx_GLEN,905141,2.762,GBP,BBG001MM1KV4
2,Global-Strategies,Kingfisher_LondonStockEx_KGF,1362038,2.276,GBP,BBG000BKH1W6
3,Global-Strategies,BurfordCapital_LondonStockEx_BUR,853486,14.060,GBP,BBG000PN88Q7
4,Global-Strategies,EKFDiagnostics_LondonStockEx_EKF,925925,0.270,GBP,BBG000BVNBN3


In [19]:
load_holdings(holdings=holdings, 
              scope=equities_scope, 
              code=portfolio_code, 
              holdings_effective_date=holdings_effective_date, 
              instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: equities_system_eaca
Code: Global-Strategies
Holdings Effective From: 2019-04-14 15:17:38.549387+00:00
Holdings Created On: 2019-04-16 15:17:43.540965+00:00



In [22]:
holdings = pd.read_csv('data/multiplesystems-holdings-options.csv')
holdings.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi
0,Global-Strategies,October 19 Calls on AMZN US,100,24528,USD,BBG00NFXK409
1,Global-Strategies,October 19 Puts on AAPL US,150,383,USD,BBG00NBRV912
2,Global-Strategies,September 19 Calls on BP/ LN,124,140,GBP,BBG00M2Z8958


In [28]:
load_holdings(holdings=holdings, 
              scope=options_scope, 
              code=portfolio_code, 
              holdings_effective_date=holdings_effective_date, 
              instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: options_system_2e06
Code: Global-Strategies
Holdings Effective From: 2019-04-14 15:17:38.549387+00:00
Holdings Created On: 2019-04-16 15:31:58.377934+00:00



## 4) Load your Transactions

## 5) Load your End of Day Positions

## 6) Perform a Bi-Temporal Reconciliation 

## 7) Group the Sources to Create an Overall View of your Fund

## 8) Add in a Fourth Source from your Fund Accountant

## 9) Reconcile your Overall View of your Fund against your Third Party Accountant